In [3]:
# Now lets download the trained model from the last notebook using joblib
import joblib

# Load the trained model from last notebook
model = joblib.load('/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl')

In [35]:
import shap
import lime
import lime.lime_tabular
import joblib
import numpy as np
import pandas as pd
import xgboost
import xgboost
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder


In [ ]:
data =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XAIentists/Data/Processed/compas-scores-two-years-processed.csv')

# Drop the two target columns
X = data.drop('two_year_recid', axis=1)
y = data['two_year_recid']

#model = joblib.load('../Trained Models/xgboost_model.pkl')

model = joblib.load("/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl", mmap_mode="r")

from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, max_categories=10)

# Fit only on training data
X_train_encoded = encoder.fit_transform(X_train)

# Apply the same transformation to test data (no .fit, only .transform!)
X_test_encoded = encoder.transform(X_test)

# Convert to DataFrame
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())

# Ensure X_test has the same columns as X_train
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

print("X_train_encoded shape:", X_train_encoded.shape)  # Should match the model's expectation
print("X_test_encoded shape:", X_test_encoded.shape)    # Should be the same as X_train

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
# Train the model on the correctly encoded data
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_encoded, y_train)

# Now the model can correctly predict using X_test_encoded
#predictions = model.predict(X_test_encoded)

explainer = lime.lime_tabular.LimeTabularExplainer(
    X_test_encoded.values,
    feature_names=X_train_encoded.columns.tolist(),
    class_names=["No Recidivism", "Recidivism"],
    mode="classification"
)

# Select a sample instance from the test set
i = 5  # Choose any index
instance = X_test_encoded.iloc[[i]]
X_test = X_test[X_train.columns]  # Ensure X_test has the same columns as X_train

# Convert instance to a DataFrame with the correct column names
instance_for_lime = pd.DataFrame([instance.iloc[0].values], columns=X_train_encoded.columns)

# Now pass this to LIME
exp = explainer.explain_instance(instance_for_lime.iloc[0].values, model.predict_proba, num_features=10)


# Print prediction
prediction = model.predict(instance)
print("instance:", instance)
print("prediction:", prediction)

# Generate explanation
#exp = explainer.explain_instance(instance.iloc[0].values, model.predict_proba, num_features=10)

# Print the explanation
print(exp.as_list())

# Visualize explanation
exp.show_in_notebook()

In [ ]:
print(model.predict_proba(instance))

In [ ]:
print(X_train.shape)  # Check the shape of the dataset

In [ ]:
# For numeric columns, you can fill missing values with the mean or median
#X_train['r_days_from_arrest'].fillna(X_train['r_days_from_arrest'].mean(), inplace=True)
X_train['r_days_from_arrest'] = X_train['r_days_from_arrest'].fillna(X_train['r_days_from_arrest'].mean())
#X_train['r_charge_desc'].fillna(X_train['r_charge_desc'].mean(), inplace=True)
X_train['r_charge_desc'] = X_train['r_charge_desc'].fillna(X_train['r_charge_desc'].mean())
#X_train['vr_charge_desc'].fillna(X_train['vr_charge_desc'].mean(), inplace=True)
X_train['vr_charge_desc'] = X_train['vr_charge_desc'].fillna(X_train['vr_charge_desc'].mean())
#X_train['r_length_of_stay'].fillna(X_train['r_length_of_stay'].mean(), inplace=True)
X_train['r_length_of_stay'] = X_train['r_length_of_stay'].fillna(X_train['r_length_of_stay'].mean())

In [57]:
# Drop rows with any missing values
X_train.dropna(inplace=True)

In [ ]:
print(X_train.isna().sum())  # Check for missing values in the dataset

print(X_train.shape)  # Check if the number of rows and columns have changed

In [ ]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

label_encoder = LabelEncoder()
X_train['sex'] = label_encoder.fit_transform(X_train['sex'])
X_train['age_cat'] = label_encoder.fit_transform(X_train['age_cat'])
X_train['race'] = label_encoder.fit_transform(X_train['race'])
X_train['c_charge_degree'] = label_encoder.fit_transform(X_train['c_charge_degree'])
X_train['r_charge_degree'] = label_encoder.fit_transform(X_train['r_charge_degree'])
X_train['vr_charge_degree'] = label_encoder.fit_transform(X_train['vr_charge_degree'])

dmatrix = xgb.DMatrix(X_train, enable_categorical=True)

# Create SHAP explainer
explainer = shap.TreeExplainer(model)  # Random Forest is tree-based

X_train['sex'] = X_train['sex'].astype('category')

# Compute SHAP values
shap_values = explainer.shap_values(X_train)
#shap_values = explainer.shap_values(dmatrix)

# Summary plot (global feature importance)
shap.summary_plot(shap_values[1], X_test, feature_names=feature_names)  # Class 1 (Recidivism)

# Explanation for a single instance
shap.force_plot(explainer.expected_value[1], shap_values[1][i], X_test.iloc[i], matplotlib=True)

# Initialize JavaScript visualization to understand how the model makes decisions (for each feature)
shap.initjs()
shap.force_plot(shap_values[1][0], X_test[0], feature_names=feature_names)

In [ ]:
# Check the data types of all columns
print(X_train.dtypes)

In [ ]:
import joblib

# Load the trained model
model = joblib.load("/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl")

# Get feature names
expected_features = model.feature_names_in_
print("Model was trained with these features:", expected_features)
print("Number of features used in training:", len(expected_features))
print("Model expects:", model.n_features_in_, "features")  # Expected feature count

# Ensure X_test_encoded has the same columns as in the trained model
X_test_encoded = X_test_encoded.reindex(columns=expected_features, fill_value=0)

print("Final shape of X_test_encoded:", X_test_encoded.shape)  # Should be (n_samples, 45)